# Healthcare Hack Nights: Part I

In [ ]:
## RNA-Seq
Questions that can be answered by RNA-seq:
    - What genes are differentially expressed between group samples?
    - How does gene expression change across time or conditions? (eg, in benign vs malignant tumors)
    - What pathways or processes are enriched under a condition?

In [2]:
import pandas as pd

In [4]:
lihc = pd.read_csv('../lihc_rnaseq.csv')

In [5]:
lihc.head()

,bcr_patient_barcode,?|100130426,?|100133144,?|100134869,?|10357,?|10431,?|136542,?|155060,?|26823,?|280660,...,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009,psiTPTE22|387590,tAKR|389932
0,TCGA-2V-A95S-01A-11R-A37K-07,0.0,1.5051,3.7074,90.1124,1017.1038,0.0,141.3911,0.6516,0.0000,...,24.7597,273.6602,794.2662,18.2440,499.1041,3172.5037,890.0472,510.1808,3.9094,6.5157
1,TCGA-2Y-A9GS-01A-12R-A38B-07,0.0,26.4120,2.6663,71.0054,639.2311,0.0,122.7206,1.4786,0.0000,...,68.5067,632.8241,1153.7703,71.4638,1000.4929,5301.1336,755.5446,860.5224,6.4071,482.9966
2,TCGA-2Y-A9GT-01A-11R-A38B-07,0.0,0.0000,4.4833,95.5122,742.4344,0.0,95.0460,1.7933,0.8967,...,46.6263,1219.4575,1133.3782,12.5532,1289.3970,3219.0092,860.7935,523.6494,14.3466,83.3894
3,TCGA-2Y-A9GU-01A-11R-A38B-07,0.0,5.7222,5.1216,61.6679,1186.9807,0.0,280.2709,0.8341,0.0000,...,18.3511,285.2758,1150.2786,9.1755,941.7437,3092.9899,1339.6283,343.6655,2.5024,2.5024
4,TCGA-2Y-A9GV-01A-11R-A38B-07,0.0,11.4975,5.4230,104.4670,878.1726,0.0,282.5719,0.0000,0.0000,...,41.4552,999.1540,1631.9797,4.2301,1380.7107,2902.7073,575.2961,665.8206,2.5381,119.2893


In [16]:
# Get ids

pd.Series(lihc.columns.values[1:]).apply(lambda x: x.split('|')[1])

0        100130426
1        100133144
2        100134869
3            10357
4            10431
           ...    
20526         7791
20527        23140
20528        26009
20529       387590
20530       389932
Length: 20531, dtype: object

In [ ]:
# TODO
# Map gene ids with GO attributes

# go_map <- getBM(attributes = c('entrezgene_id', 'hgnc_symbol', 
#                      'go_id', 'name_1006','go_linkage_type'),
#       filters = 'entrezgene_id', 
#       values = ids, 
#       mart = ensembl)
# head(go_map)

In [ ]:
# TODO
# Examine gene counts

## TCGA
For the RNA-Seq analysis workflow, we obtain the raw FASTQ sequencing files from the sequencing facility. We assess the quality of our sequence reads for each sample, then determine from where on the genome the reads originated by performing alignment. We will use the information about where the reads align to generate the count matrix, which we will be using to start the differential expression analysis.



In [ ]:
Differential gene expression analysis
- https://genomebiology.biomedcentral.com/articles/10.1186/s13059-016-0881-8
- http://bioconductor.org/packages/release/bioc/html/DESeq.htmlb
- https://nbviewer.jupyter.org/github/maayanlab/Zika-RNAseq-Pipeline/blob/master/Zika.ipynbb

In [ ]:
## Load expression matrix
expr_df = pd.read_csv(os.path.join(os.environ['WORKDIR'], 'repCpmMatrix_featureCounts.csv'))
expr_df = expr_df.set_index(expr_df.columns[0])
expr_df.head()

In [ ]:
## Filter out non-expressed genes
expr_df = expr_df.loc[expr_df.sum(axis=1) > 0, :]
print expr_df.shape

## Filter out lowly expressed genes
mask_low_vals = (expr_df > 0.3).sum(axis=1) > 2
expr_df = expr_df.loc[mask_low_vals, :]

print expr_df.shape

In [ ]:
meta_df = pd.read_csv(os.path.join(os.environ['WORKDIR'], 'SraRunTable.txt'), sep='\t').set_index('Run_s')
print meta_df.shape
# re-order the index to make it the same with expr_df
meta_df = meta_df.ix[expr_df.columns]
meta_df

## PCA Plots
- Generate PCA plots to observe whether samples cluster as expected (controls with controls and treatments with treatments)

In [ ]:
import RNAseq

In [ ]:
# plot PCA
%matplotlib inline
RNAseq.PCA_plot(expr_df.values, meta_df['infection_status_s'], 
         standardize=2, log=True, 
         show_text=False, sep=' ', legend_loc='upper right')

## Differential Gene Expression
Now we are ready to identify the differentially expressed genes between the two sets of samples: control vs. treatment. We will achieve this using the Characteristic Direction method[6](#ref6) that we developed and published in BMC Bioinformatics in 2014.

An implementation in Python of the Characteristic Direction method can be downloaded and installed from here: https://github.com/wangz10/geode.
    


In [ ]:
import geode
d_platform_cd = {} # to top up/down genes
cd_results = pd.DataFrame(index=expr_df.index)

sample_classes = {}
for layout in meta_df['LibraryLayout_s'].unique():
    ## make sample_class 
    sample_class = np.zeros(expr_df.shape[1], dtype=np.int32)
    sample_class[meta_df['LibraryLayout_s'].values == layout] = 1
    sample_class[(meta_df['LibraryLayout_s'].values == layout) & 
                 (meta_df['infection_status_s'].values == 'Zika infected')] = 2
    platform = d_layout_platform[layout]
    sample_classes[platform] = sample_class

sample_classes['combined'] = sample_classes['MiSeq'] + sample_classes['NextSeq 500']
print sample_classes

for platform, sample_class in sample_classes.items():
    cd_res = geode.chdir(expr_df.values, sample_class, expr_df.index, 
                      gamma=.5, sort=False, calculate_sig=False)
    cd_coefs = np.array(map(lambda x: x[0], cd_res))
    cd_results[platform] = cd_coefs
    
    # sort CD in by absolute values in descending order
    srt_idx = np.abs(cd_coefs).argsort()[::-1]
    cd_coefs = cd_coefs[srt_idx][:600]
    sorted_DEGs = expr_df.index[srt_idx][:600]
    # split up and down
    up_genes = dict(zip(sorted_DEGs[cd_coefs > 0], cd_coefs[cd_coefs > 0]))
    dn_genes = dict(zip(sorted_DEGs[cd_coefs < 0], cd_coefs[cd_coefs < 0]))
    d_platform_cd[platform+'-up'] = up_genes
    d_platform_cd[platform+'-dn'] = dn_genes

print cd_results.head()

In [ ]:
## Check the cosine distance between the two signatures
from scipy.spatial.distance import cosine
from itertools import combinations
for col1, col2 in combinations(cd_results.columns, 2):
    print col1, col2, cosine(cd_results[col1], cd_results[col2])

In [ ]:
## Prepare count matrices
expect input data as obtained, e.g., from RNA-seq or another high-throughput sequencing experiment, in the form of a matrix of integer values. The value in the i-th row and the j-th column of the matrix tells how many reads (or fragments, for paired-end RNA-seq) have been assigned to gene i in sample j. Analogously, for other types of assays, the rows of the matrix might correspond e.g., to binding regions (with ChIP-Seq), species of bacteria (with metagenomic datasets), or peptide sequences (with quantitative mass spectrometry).

The values in the matrix should be counts of sequencing reads/fragments. This is important for DESeq2’s statistical model to hold, as only counts allow assessing the measurement precision correctly. It is important to never provide counts that were pre-normalized for sequencing depth/library size, as the statistical model is most powerful when applied to un-normalized counts, and is designed to account for library size differences internally.

## Align Reads to reference genome
The computational analysis of an RNA-seq experiment begins from the FASTQ files that contain the nucleotide sequence of each read and a quality score at each position. These reads must first be aligned to a reference genome or transcriptome, or the abundances and estimated counts per transcript can be estimated without alignment, as described above. In either case, it is important to know if the sequencing experiment was single-end or paired-end, as the alignment software will require the user to specify both FASTQ files for a paired-end experiment. The output of this alignment step is commonly stored in a file format called SAM/BAM.


## Define gene models

## Plot counts

## PCA Plot 

## Differential Expression Analysis
## Gene Clustering





In [ ]:
# Load expression matrix


In [ ]:
from diffexp.py_deseq import py_DESeq2

dds = py_DESeq2(count_matrix = df,
               design_matrix = sample_df,
               design_formula = '~ sample',
               gene_column = 'id') # <- telling DESeq2 this should be the gene ID column
    
dds.run_deseq() 
dds.get_deseq_result()
res = dds.deseq_result 
res.head()

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(res.log2FoldChange, -np.log2(res.padj))

In [2]:
from dgeclust import CountData, SimulationManager

ModuleNotFoundError: No module named 'dgeclust'